# SNF with regression to remove s100b batch effect
s100b is a biological covariate that causes noise in clustering.\
removeBatchEffect from limma was applied to all three modalities in R. This fits a linear model to each feature in the dataset using the covariates we want to remove. SNF is run of residuals from that linear model instead of the original data.

In [1]:
import os
import numpy as np
import pandas as pd
import patsy
import scipy.stats as stats

import snf
from snf import metrics
from sklearn.cluster import spectral_clustering
from sklearn.metrics import v_measure_score
from sklearn.metrics.cluster import normalized_mutual_info_score

import matplotlib.pyplot as plt
import seaborn as sns

### Load and format data

In [2]:
def load_data():
    prot = pd.read_csv('/nfs/latdata/velina/for_bridget/s100b_corrected_omics/proteomics_182_s100b_corrected.csv',
                        index_col=0)
    tran = pd.read_csv('/nfs/latdata/velina/for_bridget/s100b_corrected_omics/rna_182_s100b_corrected.csv',
                        index_col=0)
    epig = pd.read_csv('/nfs/latdata/velina/for_bridget/s100b_corrected_omics/atac_182_s100b_corrected.csv',
                        index_col=0)
    # Metadata
    meta = pd.read_csv('/nfs/answer/fraenkel_internal/current_omics/proteomics/full_prot_metadata203_03082023.csv',
                        error_bad_lines=False)
    meta = meta.set_index('Unnamed: 0')
    
    return [prot, tran, epig, meta]

In [3]:
# Switch rows and columns to match snf format
def transpose(data):
    data_return = []
    for d in data[:-1]:
        data_return.append(d.T)
    data_return.append(data[-1])
    
    return data_return

In [4]:
# Keep only samples present in all datasets
def filter_and_sort(data):
    
    meta = data[-1]
    epig = data[-2]
    
    meta = meta[meta.index.isin(epig.index)]

    data_return = []
    
    for d in data:
        d = d[d.index.isin(meta.index)]
        data_return.append(d.sort_index())
    
    return data_return

In [5]:
data = load_data()
data = transpose(data)
prot, tran, epig, meta = filter_and_sort(data)

### Label

In [6]:
# Case
actual_labels = meta['Case'].map({'CASE': 1, 'CTRL': 0}).array

### Remove batch effect
Don't run. Could not get this to work in Python.

In [7]:
'''
https://github.com/chichaumiau/removeBatcheffect

pheno: the metadata as a table
exprs: the expression profile, the rows are the genes and the columns are the cells.
covariate_formula: is an expression of the variance you want to keep. (columns in the metadata table).
design_formula: is an expression of the variance you want to remove. i.e. the batch.
'''
def limma(pheno, exprs, covariate_formula, design_formula='1', rcond=1e-8):
    design_matrix = patsy.dmatrix(design_formula, pheno)
    rowsum = design_matrix.sum(axis=1) -1
    design_matrix=(design_matrix.T+rowsum).T

    design_batch = design_matrix
    coefficients, res, rank, s = np.linalg.lstsq(design_batch, exprs, rcond=rcond)
    beta = coefficients[-design_matrix.shape[1]:]
    return exprs - design_matrix.dot(beta)

def removeBatchEffect(cov, meta, expr_data):
    meta[cov] = meta[cov].fillna(0)

    regressed_data = []
    for df in expr_data:
        regressed_data.append(limma(meta, df, meta.drop(cov, axis=1), cov))
    return regressed_data

In [8]:
# DON'T RUN
'''
expr_data = [prot_raw, prot_norm, tran_raw, tran_norm, epig_raw, epig_norm]
regressed_data = removeBatchEffect('s100b', meta, expr_data)
prot_raw_reg, prot_norm_reg, tran_raw_reg, tran_norm_reg, epig_raw_reg, epig_norm_reg = regressed_data
prot_norm_reg
'''

"\nexpr_data = [prot_raw, prot_norm, tran_raw, tran_norm, epig_raw, epig_norm]\nregressed_data = removeBatchEffect('s100b', meta, expr_data)\nprot_raw_reg, prot_norm_reg, tran_raw_reg, tran_norm_reg, epig_raw_reg, epig_norm_reg = regressed_data\nprot_norm_reg\n"

### Apply SNF

In [9]:
def apply_snf(prot, tran, epig):
    '''
    Apply SNF to data.
    Args:
        prot: DataFrame of normalized proteomics data
        tran: DataFrame of normalized transcriptomics data
        epig: DataFrame of normalized epigenomics data
    Returns:
        best: best number of clusters by spectral clustering
        affinity_networks: array with coefficients of similarity
            between each pair of patients for a single modality
        fused_network: array with coefficients of fused similarity
            between each pair of patients
        fused_labels: array of label of 0/1 for each patient,
            determined by SNF and spectral clustering
    '''
    data = [prot, tran, epig]
    
    # K: num of nearest neighbors to consider when constructing affinity matrix
        # good value for K is sqrt(N)
    # mu: scaling factor that weights affinity matrix
    affinity_networks = snf.make_affinity(data, metric='euclidean', K=14, mu=0.5)
    
    # Run SNF algorithm
    fused_network = snf.snf(affinity_networks, K=14)
    
    # Estimate the number of clusters in the data via the “eigengap” method
    best, second = snf.get_n_clusters(fused_network)
    
    # Get labels from clusters
    fused_labels = spectral_clustering(fused_network, n_clusters=best)
    
    return best, affinity_networks, fused_network, fused_labels

# With proteomics data normalized by code
#best, fused_network, fused_labels = apply_snf(prot_z, tran_z, epig_z)
# With already normalized proteomics data
best, affinity_networks, fused_network, fused_labels = apply_snf(prot, tran, epig)
best

2

In [10]:
fused_labels

array([1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0], dtype=int32)

In [11]:
def score(best, affinity_networks, fused_network, fused_labels, actual_labels):
    '''
    Scores to evaluate performance of SNF
    Returns:
        scores_dict: dictionary of key=score name, value=score value
            for v_measure_score, nmi, and silhouette score
    '''
    # v_measure_score ranges from 0 to 1, where 1 indicates perfect overlap between the derived and true labels and 0 indicates no overlap
    v_measure = v_measure_score(fused_labels, actual_labels)
    
    # Generate cluster labels from the individual affinity matrices
    labels = [actual_labels, fused_labels]
    for arr in affinity_networks:
        labels += [spectral_clustering(arr, n_clusters=best)]
        
    '''
    Normalized mutual information score (NMI) between the labels
    generated by SNF and the ones we just obtained.
    0 indicates no overlap and 1 indicates a perfect correspondence
    between the two sets of labels.
    '''
    nmi = normalized_mutual_info_score(actual_labels, fused_labels)
    '''
    Silhouette score
    Range from -1 to 1, where -1 indicates a poor clustering solution
    and 1 indicates a fantastic solution.
    '''
    np.fill_diagonal(fused_network, 0)
    sil = metrics.silhouette_score(fused_network, fused_labels)
    
    return v_measure, nmi, sil

scores = score(best, affinity_networks, fused_network, fused_labels, actual_labels)
scores_dict = {'v_measure_score': scores[0], 'nmi': scores[1], 'silhouette score': scores[2]}
scores_dict

{'v_measure_score': 0.0013474267406975572,
 'nmi': array([[1.00000000e+00, 1.34742674e-03, 6.58543363e-05, 6.54405632e-05,
         3.22690598e-04],
        [1.34742674e-03, 1.00000000e+00, 1.18091680e-01, 4.44573059e-01,
         2.04302618e-02],
        [6.58543363e-05, 1.18091680e-01, 1.00000000e+00, 2.02395310e-02,
         1.18894996e-02],
        [6.54405632e-05, 4.44573059e-01, 2.02395310e-02, 1.00000000e+00,
         4.91343460e-03],
        [3.22690598e-04, 2.04302618e-02, 1.18894996e-02, 4.91343460e-03,
         1.00000000e+00]]),
 'silhouette score': 0.1812458527087758}

### Network visualization
Visualize connections within/between clusters generated by SNF.

In [12]:
import networkx as nx 
from pyvis import network as net

import matplotlib.cm as cm
import matplotlib as mpl

In [13]:
def add_node_colors(network, col_attribute, cmap_name="Accent"):
    """
    Function to add node colors based on existing node attribute.
    
    network: networkx object
    col_attribute: Name of attribute to use for determining node colors
    cmap_name: Name of matplotlib colormap to use (default: 'Accent')
    """
    source_att = nx.get_node_attributes(network, col_attribute)
    
    # Get colors
    cmap = cm.get_cmap(cmap_name, len(set(source_att.values())))    
    source_dict = dict(zip(list(set(source_att.values())),
                           np.arange(len(set(source_att.values())))))
    node_colors = {n:mpl.colors.rgb2hex(cmap(source_dict[v]), keep_alpha=True) for (n, v) in source_att.items()}
    # Set colors
    nx.set_node_attributes(network, node_colors, 'color')

In [14]:
n = len(fused_network)
fused_network_vis = fused_network.copy()

# Set all entries below 75% quartile to 0 to avoid too many network connections
cutoff = np.percentile(fused_network_vis, 75)
for row in fused_network_vis:
    row[row < cutoff] = 0
fused_network_vis

array([[0.        , 0.        , 0.        , ..., 0.00704014, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.0060678 ,
        0.        ],
       ...,
       [0.00704014, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.0060678 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [15]:
# Create networkx object
nt = nx.from_numpy_matrix(fused_network_vis)
nt = nx.relabel_nodes(nt, dict(zip(np.arange(n), prot.index.values)))

# Set some attributes from metadata
meta_net = meta.copy()
meta_net['fused_labels'] = fused_labels
nx.set_node_attributes(nt, meta_net.loc[:, ['Case', 'Sex.x', 'fused_labels', ]].to_dict('index'))

# Set node colors 
add_node_colors(nt, col_attribute='fused_labels')
nt.nodes['NEUAB000NKC']

{'Case': 'CASE', 'Sex.x': 'Female', 'fused_labels': 1, 'color': '#666666ff'}

In [16]:
# Subgraph for ease of visualization
sub_nt = nt.subgraph(list(nt.nodes)[:100])

ntw = net.Network('750px', '750px', notebook=True)
ntw.toggle_physics(False)
ntw.from_nx(sub_nt)

ntw.show('nx.html')

nx.html
